In [2]:
using POMDPs
using Random # for AbstractRNG
using POMDPModelTools

In [3]:
struct ChainMDP <: MDP{Int, Symbol}
    len::Int
    p_success::Float64
    discount::Float64
end


In [4]:
function POMDPs.generate_s(p::ChainMDP, s::Int, a::Symbol, rng::AbstractRNG)
    if a == :right
        success = min(s+1, p.len)
        failure = max(s-1, 1)
    else # a == :left
        success = max(s-1, 1)
        failure = min(s+1, p.len)
    end
    if s + 1 == p.len
        return p.len 
    elseif  s == 2
        return 1
    end
    return rand(rng) < p.p_success ? success : failure
end


In [5]:
function POMDPs.reward(p::ChainMDP, s::Int, a::Symbol)
    if s == 2
        return 10
    end
    if s + 1 == p.len
        return -10
    end
    if s == 0 || s == p.len
        return 0
    end
    return -1
end

In [6]:
POMDPs.initialstate_distribution(m::ChainMDP) = Deterministic(4)

In [7]:
function POMDPs.isterminal(p::ChainMDP, s::Int)
    if s == 1
        return true
    end
    if s == p.len
        return true
    end
    return false
end        

In [8]:
using POMDPSimulators
using POMDPPolicies

ChainMDP() = ChainMDP(6+2,1.0,.9)
m = ChainMDP()

# policy that maps every input to a right action
policy = FunctionPolicy(s->:right)

for (s, a, r) in stepthrough(m, policy, "s,a,r", max_steps=10)
    @show s
    @show a
    @show r
    render(m, (s,a,r))
    println()
end


s = 4
a = :right
r = -1

s = 5
a = :right
r = -1

s = 6
a = :right
r = -1

s = 7
a = :right
r = -10



In [9]:
using POMDPSimulators
using POMDPPolicies

ChainMDP() = ChainMDP(6+2,1.0,.9)
m = ChainMDP()

# policy that maps every input to a left action
policy = FunctionPolicy(s->:left)

function POMDPs.initialstate_distribution(m::ChainMDP)
    return Deterministic(4)
end
for (s, a, r) in stepthrough(m, policy, "s,a,r", max_steps=10)
    # @show s
    # @show a
    # @show r
    render(m, (s,a,r))
    println("s,a,r:($s,$a,$r)")
end


s,a,r:(4,left,-1)
s,a,r:(3,left,-1)
s,a,r:(2,left,10)


In [10]:

ChainMDP() = ChainMDP(6+2,1.0,.9)
m = ChainMDP()

# policy that maps every input to a right action
policy = FunctionPolicy(s->:right)

function POMDPs.initialstate_distribution(m::ChainMDP)
    return Deterministic(4)
end
for (s, a, r) in stepthrough(m, policy, "s,a,r", max_steps=10)
    # @show s
    # @show a
    # @show r
    render(m, (s,a,r))
    println("s,a,r:($s,$a,$r)")
end


s,a,r:(4,right,-1)
s,a,r:(5,right,-1)
s,a,r:(6,right,-1)
s,a,r:(7,right,-10)


In [12]:
n_agents = 5
agents = Any[]
mdps = Any[]
n_states = 10
n_actions = 2
epochs = 10
H = 10
actions = [1, 2]
action_map = Dict(1 => :left, :2 => :right)
rev_action_map = Dict(:left => 1, :right => 2)
states = 1:(n_states+2)
# setup agents
Q_tables = []
N_tables = zeros((n_agents, n_states+2, n_actions))
print(N_tables)
policies = []
# policy that maps every input to a right action
for i in 1:n_agents
    push!(Q_tables, Dict{Int32,Float32}())
    push!(policies, s->:right)
    push!(mdps, ChainMDP(n_states+2,.9,.9))
end
# policy = s->:right

function update_Q(Q_table, s, a, r, sp, t)
    println("s:$s, a:$a, sp:$sp, r:$r, t:$t")
end
function POMDPs.initialstate_distribution(m::ChainMDP)
    return Deterministic(Int64((n_states+2)/2))
end
function run_chain!(policies, mdps, update_Q, n_agents, n_states,
                    Q_tables, N_tables, epochs, steps)
    for e in 1:epochs
        agents = []
        for i in 1:n_agents
            m = mdps[i]
            push!(agents, Iterators.Stateful(stepthrough(m, FunctionPolicy(policies[i]), "s,a,r,sp,t", max_steps=steps)))
        end

        println("epoch: $e")
        done = false
        t = 0
        while ! done
           done = true
            
           for i in 1:n_agents
                if isempty(agents[i])
                    println("agent $i is done")
                    continue
                end
                res = popfirst!(agents[i])
                #println("before update: N_table: $N_tables")
                N_tables[i,res[:s],rev_action_map[res[:a]]] += 1
                #println("after update:  N_table: $N_tables")
                # println("update Q")
                update_Q(Q_tables[i],res...)
                println("print agent $i result: $res")
                t = res[:t]
          
           end
           println("t:$t")
           for i in 1:n_agents
                if ! isempty(agents[i])
                    done = false
                end
            end
        end
    end
end
run_chain!(policies, mdps, update_Q, n_agents,  n_states, Q_tables, N_tables, epochs, H)

[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]

[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]epoch: 1
s:6, a:right, sp:7, r:-1, t:1
print agent 1 result: (s = 6, a = :right, r = -1, sp = 7, t = 1)
s:6, a:right, sp:7, r:-1, t:1
print agent 2 result: (s = 6, a = :right, r = -1, sp = 7, t = 1)
s:6, a:right, sp:7, r:-1, t:1
print agent 3 result: (s = 6, a = :right, r = -1, sp = 7, t = 1)
s:6, a:right, sp:7, r:-1, t:1
print agent 4 result: (s = 6, a = :right, r = -1, sp = 7, t = 1)
s:6, a:right, sp:7, r:-1, t:1
print agent 5 result: (s = 6, a = :right, r = -1, sp = 7, t = 1)
t:1
s:7, a:right, sp:6, 

print agent 1 result: (s = 11, a = :right, r = -10, sp = 12, t = 6)
s:9, a:right, sp:10, r:-1, t:6
print agent 2 result: (s = 9, a = :right, r = -1, sp = 10, t = 6)
s:11, a:right, sp:12, r:-10, t:6
print agent 3 result: (s = 11, a = :right, r = -10, sp = 12, t = 6)
s:11, a:right, sp:12, r:-10, t:6
print agent 4 result: (s = 11, a = :right, r = -10, sp = 12, t = 6)
s:9, a:right, sp:10, r:-1, t:6
print agent 5 result: (s = 9, a = :right, r = -1, sp = 10, t = 6)
t:6
agent 1 is done
s:10, a:right, sp:11, r:-1, t:7
print agent 2 result: (s = 10, a = :right, r = -1, sp = 11, t = 7)
agent 3 is done
agent 4 is done
s:10, a:right, sp:11, r:-1, t:7
print agent 5 result: (s = 10, a = :right, r = -1, sp = 11, t = 7)
t:7
agent 1 is done
s:11, a:right, sp:12, r:-10, t:8
print agent 2 result: (s = 11, a = :right, r = -10, sp = 12, t = 8)
agent 3 is done
agent 4 is done
s:11, a:right, sp:12, r:-10, t:8
print agent 5 result: (s = 11, a = :right, r = -10, sp = 12, t = 8)
t:8
epoch: 4
s:6, a:right, sp:7,

s:10, a:right, sp:11, r:-1, t:5
print agent 3 result: (s = 10, a = :right, r = -1, sp = 11, t = 5)
s:10, a:right, sp:11, r:-1, t:5
print agent 4 result: (s = 10, a = :right, r = -1, sp = 11, t = 5)
s:10, a:right, sp:11, r:-1, t:5
print agent 5 result: (s = 10, a = :right, r = -1, sp = 11, t = 5)
t:5
s:9, a:right, sp:10, r:-1, t:6
print agent 1 result: (s = 9, a = :right, r = -1, sp = 10, t = 6)
s:11, a:right, sp:12, r:-10, t:6
print agent 2 result: (s = 11, a = :right, r = -10, sp = 12, t = 6)
s:11, a:right, sp:12, r:-10, t:6
print agent 3 result: (s = 11, a = :right, r = -10, sp = 12, t = 6)
s:11, a:right, sp:12, r:-10, t:6
print agent 4 result: (s = 11, a = :right, r = -10, sp = 12, t = 6)
s:11, a:right, sp:12, r:-10, t:6
print agent 5 result: (s = 11, a = :right, r = -10, sp = 12, t = 6)
t:6
s:10, a:right, sp:11, r:-1, t:7
print agent 1 result: (s = 10, a = :right, r = -1, sp = 11, t = 7)
agent 2 is done
agent 3 is done
agent 4 is done
agent 5 is done
t:7
s:11, a:right, sp:12, r:-10

print agent 4 result: (s = 9, a = :right, r = -1, sp = 10, t = 4)
s:9, a:right, sp:10, r:-1, t:4
print agent 5 result: (s = 9, a = :right, r = -1, sp = 10, t = 4)
t:4
s:10, a:right, sp:9, r:-1, t:5
print agent 1 result: (s = 10, a = :right, r = -1, sp = 9, t = 5)
s:10, a:right, sp:11, r:-1, t:5
print agent 2 result: (s = 10, a = :right, r = -1, sp = 11, t = 5)
s:10, a:right, sp:11, r:-1, t:5
print agent 3 result: (s = 10, a = :right, r = -1, sp = 11, t = 5)
s:10, a:right, sp:11, r:-1, t:5
print agent 4 result: (s = 10, a = :right, r = -1, sp = 11, t = 5)
s:10, a:right, sp:11, r:-1, t:5
print agent 5 result: (s = 10, a = :right, r = -1, sp = 11, t = 5)
t:5
s:9, a:right, sp:10, r:-1, t:6
print agent 1 result: (s = 9, a = :right, r = -1, sp = 10, t = 6)
s:11, a:right, sp:12, r:-10, t:6
print agent 2 result: (s = 11, a = :right, r = -10, sp = 12, t = 6)
s:11, a:right, sp:12, r:-10, t:6
print agent 3 result: (s = 11, a = :right, r = -10, sp = 12, t = 6)
s:11, a:right, sp:12, r:-10, t:6
prin

In [11]:
function square(x)
    return x * x
end

vals = Dict(1 => 4, 2 => 5, 3 => 6)
Dict(key=> square(val) for (key, val) in vals)

Dict{Int64,Int64} with 3 entries:
  2 => 25
  3 => 36
  1 => 16

In [12]:
A = [1 2; 3 4; 5 6]
B = A .+ 6
C = B .+ 6
D = hcat([A , B , C])

3×1 Array{Array{Int64,2},2}:
 [1 2; 3 4; 5 6]      
 [7 8; 9 10; 11 12]   
 [13 14; 15 16; 17 18]

In [13]:
D[2,:]

1-element Array{Array{Int64,2},1}:
 [7 8; 9 10; 11 12]

In [14]:
function ucb_pol(Q_tables, N_tables, i, actions, s)
    # return 
    if haskey(Q_tables[i], s)
       # println("ucb_pol:s:$s, N_tables:$N_tables")
       ucbs = Dict(key => sqrt(log(sum(N_tables[i,s,:])/N_tables[i,s,key])) 
                for (key,val) in Q_tables[i][s])
       # println("ucb: $(ucbs)")
       val, idx = findmax(Dict(key => Q_tables[i][s][key] + sqrt(log(sum(N_tables[i,s,:])/N_tables[i,s,key])) 
                for (key,val) in Q_tables[i][s])) # need to test
       return action_map[idx]
    else
       act = action_map[rand(actions, 1)[1]]
       # println("selected action $act") 
       return act
    end
end

function update_Q(Q_table, s, a, r, sp, t)
    alpha=.95
    gamma=.95
    # print("s:$s, a:$a, sp:$sp, r:$r")
    Q_table[s][rev_action_map[a]] +=  alpha * (r + gamma * 
        findmax(Q_table[sp])[1] - Q_table[s][rev_action_map[a]])
    #println(Q_table)
end


update_Q (generic function with 1 method)

In [15]:
# UCB

n_states = 10
curry(f, x) = (xs...) -> f(x, xs...)
Q_tables = []
mdps = Any[]
N_tables = zeros((n_agents, n_states+2, n_actions))
policies = []
epochs = 500
theta = 10
H = 10
# Setup Q table according to MDP

n_agents = 1
agents = Any[]
n_states = 10

function POMDPs.reward(p::ChainMDP, s::Int, a::Symbol)
    if s == 2
        return 10
    end
    if s + 1 == p.len
        return -10
    end
    if s == 0 || s == p.len
        return 0
    end
    return -1
end

#setup
for i in 1:n_agents
    push!(Q_tables, Dict{Int32, Dict{Int32, Float32}}())
    for state in states
      Q_tables[i][state] = Dict{Int32, Float32}()
      # print(Q_tables[i])
      for action in actions
          
#           if state == 2
#              Q_tables[i][state][action] = theta
#           elseif state + 1 == n_states + 2
#               Q_tables[i][state][action] = -theta
#           else
#               Q_tables[i][state][action] = -1
#           end
         Q_tables[i][state][action] = 0 
      end
    end
    N_table = N_tables[i,:,:]
    push!(policies, curry(curry(curry(curry(ucb_pol, Q_tables), N_tables),i), actions))
    push!(mdps, ChainMDP(n_states+2,.9,.9))
end

run_chain!(policies, mdps, update_Q, n_agents,  n_states, Q_tables, N_tables, epochs, H)

epoch: 1
print agent 1 result: (s = 6, a = :right, r = -1, sp = 7, t = 1)
t:1
print agent 1 result: (s = 7, a = :right, r = -1, sp = 6, t = 2)
t:2
print agent 1 result: (s = 6, a = :left, r = -1, sp = 5, t = 3)
t:3
print agent 1 result: (s = 5, a = :right, r = -1, sp = 6, t = 4)
t:4
print agent 1 result: (s = 6, a = :right, r = -1, sp = 7, t = 5)
t:5
print agent 1 result: (s = 7, a = :left, r = -1, sp = 6, t = 6)
t:6
print agent 1 result: (s = 6, a = :left, r = -1, sp = 5, t = 7)
t:7
print agent 1 result: (s = 5, a = :left, r = -1, sp = 4, t = 8)
t:8
print agent 1 result: (s = 4, a = :right, r = -1, sp = 5, t = 9)
t:9
print agent 1 result: (s = 5, a = :left, r = -1, sp = 4, t = 10)
t:10
epoch: 2
print agent 1 result: (s = 6, a = :right, r = -1, sp = 7, t = 1)
t:1
print agent 1 result: (s = 7, a = :right, r = -1, sp = 6, t = 2)
t:2
print agent 1 result: (s = 6, a = :left, r = -1, sp = 5, t = 3)
t:3
print agent 1 result: (s = 5, a = :left, r = -1, sp = 4, t = 4)
t:4
print agent 1 result:

t:7
epoch: 20
print agent 1 result: (s = 6, a = :left, r = -1, sp = 5, t = 1)
t:1
print agent 1 result: (s = 5, a = :left, r = -1, sp = 6, t = 2)
t:2
print agent 1 result: (s = 6, a = :left, r = -1, sp = 5, t = 3)
t:3
print agent 1 result: (s = 5, a = :left, r = -1, sp = 4, t = 4)
t:4
print agent 1 result: (s = 4, a = :left, r = -1, sp = 3, t = 5)
t:5
print agent 1 result: (s = 3, a = :left, r = -1, sp = 2, t = 6)
t:6
print agent 1 result: (s = 2, a = :left, r = 10, sp = 1, t = 7)
t:7
epoch: 21
print agent 1 result: (s = 6, a = :left, r = -1, sp = 5, t = 1)
t:1
print agent 1 result: (s = 5, a = :left, r = -1, sp = 4, t = 2)
t:2
print agent 1 result: (s = 4, a = :left, r = -1, sp = 5, t = 3)
t:3
print agent 1 result: (s = 5, a = :left, r = -1, sp = 4, t = 4)
t:4
print agent 1 result: (s = 4, a = :left, r = -1, sp = 3, t = 5)
t:5
print agent 1 result: (s = 3, a = :left, r = -1, sp = 2, t = 6)
t:6
print agent 1 result: (s = 2, a = :right, r = 10, sp = 1, t = 7)
t:7
epoch: 22
print agent 1

t:1
print agent 1 result: (s = 5, a = :left, r = -1, sp = 4, t = 2)
t:2
print agent 1 result: (s = 4, a = :left, r = -1, sp = 3, t = 3)
t:3
print agent 1 result: (s = 3, a = :left, r = -1, sp = 4, t = 4)
t:4
print agent 1 result: (s = 4, a = :left, r = -1, sp = 3, t = 5)
t:5
print agent 1 result: (s = 3, a = :left, r = -1, sp = 2, t = 6)
t:6
print agent 1 result: (s = 2, a = :left, r = 10, sp = 1, t = 7)
t:7
epoch: 44
print agent 1 result: (s = 6, a = :left, r = -1, sp = 7, t = 1)
t:1
print agent 1 result: (s = 7, a = :right, r = -1, sp = 8, t = 2)
t:2
print agent 1 result: (s = 8, a = :right, r = -1, sp = 9, t = 3)
t:3
print agent 1 result: (s = 9, a = :left, r = -1, sp = 8, t = 4)
t:4
print agent 1 result: (s = 8, a = :right, r = -1, sp = 9, t = 5)
t:5
print agent 1 result: (s = 9, a = :right, r = -1, sp = 10, t = 6)
t:6
print agent 1 result: (s = 10, a = :left, r = -1, sp = 11, t = 7)
t:7
print agent 1 result: (s = 11, a = :right, r = -10, sp = 12, t = 8)
t:8
epoch: 45
print agent 1

t:10
epoch: 64
print agent 1 result: (s = 6, a = :right, r = -1, sp = 7, t = 1)
t:1
print agent 1 result: (s = 7, a = :left, r = -1, sp = 6, t = 2)
t:2
print agent 1 result: (s = 6, a = :left, r = -1, sp = 5, t = 3)
t:3
print agent 1 result: (s = 5, a = :left, r = -1, sp = 4, t = 4)
t:4
print agent 1 result: (s = 4, a = :left, r = -1, sp = 3, t = 5)
t:5
print agent 1 result: (s = 3, a = :left, r = -1, sp = 2, t = 6)
t:6
print agent 1 result: (s = 2, a = :left, r = 10, sp = 1, t = 7)
t:7
epoch: 65
print agent 1 result: (s = 6, a = :left, r = -1, sp = 5, t = 1)
t:1
print agent 1 result: (s = 5, a = :left, r = -1, sp = 4, t = 2)
t:2
print agent 1 result: (s = 4, a = :left, r = -1, sp = 3, t = 3)
t:3
print agent 1 result: (s = 3, a = :left, r = -1, sp = 2, t = 4)
t:4
print agent 1 result: (s = 2, a = :right, r = 10, sp = 1, t = 5)
t:5
epoch: 66
print agent 1 result: (s = 6, a = :left, r = -1, sp = 5, t = 1)
t:1
print agent 1 result: (s = 5, a = :left, r = -1, sp = 4, t = 2)
t:2
print agent

t:9
epoch: 88
print agent 1 result: (s = 6, a = :left, r = -1, sp = 5, t = 1)
t:1
print agent 1 result: (s = 5, a = :left, r = -1, sp = 4, t = 2)
t:2
print agent 1 result: (s = 4, a = :left, r = -1, sp = 5, t = 3)
t:3
print agent 1 result: (s = 5, a = :left, r = -1, sp = 6, t = 4)
t:4
print agent 1 result: (s = 6, a = :left, r = -1, sp = 5, t = 5)
t:5
print agent 1 result: (s = 5, a = :left, r = -1, sp = 4, t = 6)
t:6
print agent 1 result: (s = 4, a = :left, r = -1, sp = 3, t = 7)
t:7
print agent 1 result: (s = 3, a = :left, r = -1, sp = 4, t = 8)
t:8
print agent 1 result: (s = 4, a = :left, r = -1, sp = 3, t = 9)
t:9
print agent 1 result: (s = 3, a = :left, r = -1, sp = 2, t = 10)
t:10
epoch: 89
print agent 1 result: (s = 6, a = :left, r = -1, sp = 5, t = 1)
t:1
print agent 1 result: (s = 5, a = :left, r = -1, sp = 4, t = 2)
t:2
print agent 1 result: (s = 4, a = :left, r = -1, sp = 3, t = 3)
t:3
print agent 1 result: (s = 3, a = :left, r = -1, sp = 2, t = 4)
t:4
print agent 1 result: 

...

t:5
print agent 1 result: (s = 11, a = :left, r = -10, sp = 12, t = 6)
t:6
epoch: 403
print agent 1 result: (s = 6, a = :left, r = -1, sp = 7, t = 1)
t:1
print agent 1 result: (s = 7, a = :right, r = -1, sp = 8, t = 2)
t:2
print agent 1 result: (s = 8, a = :right, r = -1, sp = 9, t = 3)
t:3
print agent 1 result: (s = 9, a = :right, r = -1, sp = 10, t = 4)
t:4
print agent 1 result: (s = 10, a = :right, r = -1, sp = 11, t = 5)
t:5
print agent 1 result: (s = 11, a = :right, r = -10, sp = 12, t = 6)
t:6
epoch: 404
print agent 1 result: (s = 6, a = :left, r = -1, sp = 7, t = 1)
t:1
print agent 1 result: (s = 7, a = :right, r = -1, sp = 8, t = 2)
t:2
print agent 1 result: (s = 8, a = :right, r = -1, sp = 7, t = 3)
t:3
print agent 1 result: (s = 7, a = :right, r = -1, sp = 8, t = 4)
t:4
print agent 1 result: (s = 8, a = :right, r = -1, sp = 7, t = 5)
t:5
print agent 1 result: (s = 7, a = :right, r = -1, sp = 8, t = 6)
t:6
print agent 1 result: (s = 8, a = :right, r = -1, sp = 9, t = 7)
t:7
pr

t:2
print agent 1 result: (s = 4, a = :left, r = -1, sp = 3, t = 3)
t:3
print agent 1 result: (s = 3, a = :left, r = -1, sp = 2, t = 4)
t:4
print agent 1 result: (s = 2, a = :left, r = 10, sp = 1, t = 5)
t:5
epoch: 427
print agent 1 result: (s = 6, a = :left, r = -1, sp = 5, t = 1)
t:1
print agent 1 result: (s = 5, a = :left, r = -1, sp = 4, t = 2)
t:2
print agent 1 result: (s = 4, a = :left, r = -1, sp = 3, t = 3)
t:3
print agent 1 result: (s = 3, a = :left, r = -1, sp = 2, t = 4)
t:4
print agent 1 result: (s = 2, a = :right, r = 10, sp = 1, t = 5)
t:5
epoch: 428
print agent 1 result: (s = 6, a = :left, r = -1, sp = 5, t = 1)
t:1
print agent 1 result: (s = 5, a = :left, r = -1, sp = 4, t = 2)
t:2
print agent 1 result: (s = 4, a = :left, r = -1, sp = 3, t = 3)
t:3
print agent 1 result: (s = 3, a = :left, r = -1, sp = 2, t = 4)
t:4
print agent 1 result: (s = 2, a = :left, r = 10, sp = 1, t = 5)
t:5
epoch: 429
print agent 1 result: (s = 6, a = :left, r = -1, sp = 5, t = 1)
t:1
print agen

t:6
print agent 1 result: (s = 4, a = :left, r = -1, sp = 3, t = 7)
t:7
print agent 1 result: (s = 3, a = :left, r = -1, sp = 2, t = 8)
t:8
print agent 1 result: (s = 2, a = :left, r = 10, sp = 1, t = 9)
t:9
epoch: 452
print agent 1 result: (s = 6, a = :left, r = -1, sp = 5, t = 1)
t:1
print agent 1 result: (s = 5, a = :left, r = -1, sp = 6, t = 2)
t:2
print agent 1 result: (s = 6, a = :left, r = -1, sp = 5, t = 3)
t:3
print agent 1 result: (s = 5, a = :left, r = -1, sp = 4, t = 4)
t:4
print agent 1 result: (s = 4, a = :left, r = -1, sp = 3, t = 5)
t:5
print agent 1 result: (s = 3, a = :left, r = -1, sp = 2, t = 6)
t:6
print agent 1 result: (s = 2, a = :right, r = 10, sp = 1, t = 7)
t:7
epoch: 453
print agent 1 result: (s = 6, a = :left, r = -1, sp = 5, t = 1)
t:1
print agent 1 result: (s = 5, a = :left, r = -1, sp = 6, t = 2)
t:2
print agent 1 result: (s = 6, a = :left, r = -1, sp = 5, t = 3)
t:3
print agent 1 result: (s = 5, a = :left, r = -1, sp = 6, t = 4)
t:4
print agent 1 result:

print agent 1 result: (s = 4, a = :left, r = -1, sp = 3, t = 3)
t:3
print agent 1 result: (s = 3, a = :left, r = -1, sp = 2, t = 4)
t:4
print agent 1 result: (s = 2, a = :left, r = 10, sp = 1, t = 5)
t:5
epoch: 477
print agent 1 result: (s = 6, a = :left, r = -1, sp = 5, t = 1)
t:1
print agent 1 result: (s = 5, a = :left, r = -1, sp = 4, t = 2)
t:2
print agent 1 result: (s = 4, a = :left, r = -1, sp = 5, t = 3)
t:3
print agent 1 result: (s = 5, a = :left, r = -1, sp = 4, t = 4)
t:4
print agent 1 result: (s = 4, a = :left, r = -1, sp = 3, t = 5)
t:5
print agent 1 result: (s = 3, a = :left, r = -1, sp = 2, t = 6)
t:6
print agent 1 result: (s = 2, a = :right, r = 10, sp = 1, t = 7)
t:7
epoch: 478
print agent 1 result: (s = 6, a = :left, r = -1, sp = 7, t = 1)
t:1
print agent 1 result: (s = 7, a = :right, r = -1, sp = 8, t = 2)
t:2
print agent 1 result: (s = 8, a = :right, r = -1, sp = 7, t = 3)
t:3
print agent 1 result: (s = 7, a = :right, r = -1, sp = 8, t = 4)
t:4
print agent 1 result: 

t:3
print agent 1 result: (s = 3, a = :left, r = -1, sp = 2, t = 4)
t:4
print agent 1 result: (s = 2, a = :right, r = 10, sp = 1, t = 5)
t:5
epoch: 500
print agent 1 result: (s = 6, a = :left, r = -1, sp = 5, t = 1)
t:1
print agent 1 result: (s = 5, a = :left, r = -1, sp = 6, t = 2)
t:2
print agent 1 result: (s = 6, a = :left, r = -1, sp = 5, t = 3)
t:3
print agent 1 result: (s = 5, a = :left, r = -1, sp = 4, t = 4)
t:4
print agent 1 result: (s = 4, a = :left, r = -1, sp = 3, t = 5)
t:5
print agent 1 result: (s = 3, a = :left, r = -1, sp = 2, t = 6)
t:6
print agent 1 result: (s = 2, a = :left, r = 10, sp = 1, t = 7)
t:7


In [17]:

n_agents = 10
Q_tables = [deepcopy(Q_tables[1]) for i in range(1,n_agents)]
policies = [deepcopy(policies[1]) for i in range(1,n_agents)]
mdps = [deepcopy(mdps[1]) for i in range(1,n_agents)]
N_tables = zeros((n_agents, n_states+2, n_actions))
run_chain!(policies, mdps, update_Q, n_agents,  n_states, Q_tables, N_tables, epochs, H)

epoch: 1
print agent 1 result: (s = 6, a = :left, r = -1, sp = 5, t = 1)
print agent 2 result: (s = 6, a = :left, r = -1, sp = 5, t = 1)
print agent 3 result: (s = 6, a = :left, r = -1, sp = 5, t = 1)
print agent 4 result: (s = 6, a = :left, r = -1, sp = 5, t = 1)
print agent 5 result: (s = 6, a = :left, r = -1, sp = 5, t = 1)
print agent 6 result: (s = 6, a = :left, r = -1, sp = 5, t = 1)
print agent 7 result: (s = 6, a = :left, r = -1, sp = 5, t = 1)
print agent 8 result: (s = 6, a = :left, r = -1, sp = 5, t = 1)
print agent 9 result: (s = 6, a = :left, r = -1, sp = 5, t = 1)
print agent 10 result: (s = 6, a = :left, r = -1, sp = 5, t = 1)
t:1
print agent 1 result: (s = 5, a = :left, r = -1, sp = 4, t = 2)
print agent 2 result: (s = 5, a = :left, r = -1, sp = 6, t = 2)
print agent 3 result: (s = 5, a = :left, r = -1, sp = 4, t = 2)
print agent 4 result: (s = 5, a = :left, r = -1, sp = 4, t = 2)
print agent 5 result: (s = 5, a = :left, r = -1, sp = 4, t = 2)
print agent 6 result: (s =

print agent 9 result: (s = 5, a = :left, r = -1, sp = 4, t = 2)
print agent 10 result: (s = 5, a = :left, r = -1, sp = 4, t = 2)
t:2
print agent 1 result: (s = 4, a = :left, r = -1, sp = 3, t = 3)
print agent 2 result: (s = 4, a = :left, r = -1, sp = 3, t = 3)
print agent 3 result: (s = 4, a = :left, r = -1, sp = 3, t = 3)
print agent 4 result: (s = 4, a = :left, r = -1, sp = 3, t = 3)
print agent 5 result: (s = 4, a = :left, r = -1, sp = 5, t = 3)
print agent 6 result: (s = 6, a = :left, r = -1, sp = 5, t = 3)
print agent 7 result: (s = 4, a = :left, r = -1, sp = 3, t = 3)
print agent 8 result: (s = 4, a = :left, r = -1, sp = 3, t = 3)
print agent 9 result: (s = 4, a = :left, r = -1, sp = 3, t = 3)
print agent 10 result: (s = 4, a = :left, r = -1, sp = 3, t = 3)
t:3
print agent 1 result: (s = 3, a = :left, r = -1, sp = 2, t = 4)
print agent 2 result: (s = 3, a = :left, r = -1, sp = 2, t = 4)
print agent 3 result: (s = 3, a = :left, r = -1, sp = 4, t = 4)
print agent 4 result: (s = 3, 

t:4
print agent 1 result: (s = 10, a = :right, r = -1, sp = 11, t = 5)
print agent 2 result: (s = 2, a = :right, r = 10, sp = 1, t = 5)
print agent 3 result: (s = 4, a = :left, r = -1, sp = 5, t = 5)
print agent 4 result: (s = 2, a = :right, r = 10, sp = 1, t = 5)
print agent 5 result: (s = 2, a = :right, r = 10, sp = 1, t = 5)
print agent 6 result: (s = 6, a = :left, r = -1, sp = 5, t = 5)
print agent 7 result: (s = 4, a = :left, r = -1, sp = 5, t = 5)
print agent 8 result: (s = 4, a = :left, r = -1, sp = 3, t = 5)
print agent 9 result: (s = 10, a = :right, r = -1, sp = 11, t = 5)
print agent 10 result: (s = 2, a = :right, r = 10, sp = 1, t = 5)
t:5
print agent 1 result: (s = 11, a = :left, r = -10, sp = 12, t = 6)
agent 2 is done
print agent 3 result: (s = 5, a = :left, r = -1, sp = 4, t = 6)
agent 4 is done
agent 5 is done
print agent 6 result: (s = 5, a = :left, r = -1, sp = 4, t = 6)
print agent 7 result: (s = 5, a = :left, r = -1, sp = 4, t = 6)
print agent 8 result: (s = 3, a = 

agent 4 is done
print agent 5 result: (s = 11, a = :left, r = -10, sp = 12, t = 6)
agent 6 is done
agent 7 is done
agent 8 is done
agent 9 is done
agent 10 is done
t:6
agent 1 is done
agent 2 is done
print agent 3 result: (s = 2, a = :right, r = 10, sp = 1, t = 7)
agent 4 is done
agent 5 is done
agent 6 is done
agent 7 is done
agent 8 is done
agent 9 is done
agent 10 is done
t:7
epoch: 8
print agent 1 result: (s = 6, a = :left, r = -1, sp = 5, t = 1)
print agent 2 result: (s = 6, a = :left, r = -1, sp = 5, t = 1)
print agent 3 result: (s = 6, a = :left, r = -1, sp = 5, t = 1)
print agent 4 result: (s = 6, a = :left, r = -1, sp = 5, t = 1)
print agent 5 result: (s = 6, a = :left, r = -1, sp = 5, t = 1)
print agent 6 result: (s = 6, a = :left, r = -1, sp = 5, t = 1)
print agent 7 result: (s = 6, a = :left, r = -1, sp = 5, t = 1)
print agent 8 result: (s = 6, a = :left, r = -1, sp = 5, t = 1)
print agent 9 result: (s = 6, a = :left, r = -1, sp = 5, t = 1)
print agent 10 result: (s = 6, a 

print agent 8 result: (s = 5, a = :left, r = -1, sp = 4, t = 2)
print agent 9 result: (s = 5, a = :left, r = -1, sp = 4, t = 2)
print agent 10 result: (s = 5, a = :left, r = -1, sp = 6, t = 2)
t:2
print agent 1 result: (s = 4, a = :left, r = -1, sp = 3, t = 3)
print agent 2 result: (s = 4, a = :left, r = -1, sp = 3, t = 3)
print agent 3 result: (s = 6, a = :left, r = -1, sp = 5, t = 3)
print agent 4 result: (s = 8, a = :right, r = -1, sp = 9, t = 3)
print agent 5 result: (s = 4, a = :left, r = -1, sp = 3, t = 3)
print agent 6 result: (s = 4, a = :left, r = -1, sp = 3, t = 3)
print agent 7 result: (s = 4, a = :left, r = -1, sp = 3, t = 3)
print agent 8 result: (s = 4, a = :left, r = -1, sp = 3, t = 3)
print agent 9 result: (s = 4, a = :left, r = -1, sp = 3, t = 3)
print agent 10 result: (s = 6, a = :left, r = -1, sp = 5, t = 3)
t:3
print agent 1 result: (s = 3, a = :left, r = -1, sp = 2, t = 4)
print agent 2 result: (s = 3, a = :left, r = -1, sp = 2, t = 4)
print agent 3 result: (s = 5,

...

agent 3 is done
agent 4 is done
agent 5 is done
agent 6 is done
agent 7 is done
print agent 8 result: (s = 4, a = :left, r = -1, sp = 3, t = 7)
print agent 9 result: (s = 2, a = :right, r = 10, sp = 1, t = 7)
agent 10 is done
t:7
print agent 1 result: (s = 3, a = :left, r = -1, sp = 2, t = 8)
agent 2 is done
agent 3 is done
agent 4 is done
agent 5 is done
agent 6 is done
agent 7 is done
print agent 8 result: (s = 3, a = :left, r = -1, sp = 2, t = 8)
agent 9 is done
agent 10 is done
t:8
print agent 1 result: (s = 2, a = :right, r = 10, sp = 1, t = 9)
agent 2 is done
agent 3 is done
agent 4 is done
agent 5 is done
agent 6 is done
agent 7 is done
print agent 8 result: (s = 2, a = :right, r = 10, sp = 1, t = 9)
agent 9 is done
agent 10 is done
t:9
epoch: 119
print agent 1 result: (s = 6, a = :left, r = -1, sp = 5, t = 1)
print agent 2 result: (s = 6, a = :left, r = -1, sp = 5, t = 1)
print agent 3 result: (s = 6, a = :left, r = -1, sp = 5, t = 1)
print agent 4 result: (s = 6, a = :left, r 

print agent 9 result: (s = 6, a = :left, r = -1, sp = 5, t = 1)
print agent 10 result: (s = 6, a = :left, r = -1, sp = 5, t = 1)
t:1
print agent 1 result: (s = 5, a = :left, r = -1, sp = 4, t = 2)
print agent 2 result: (s = 5, a = :left, r = -1, sp = 4, t = 2)
print agent 3 result: (s = 5, a = :left, r = -1, sp = 4, t = 2)
print agent 4 result: (s = 7, a = :right, r = -1, sp = 8, t = 2)
print agent 5 result: (s = 5, a = :left, r = -1, sp = 4, t = 2)
print agent 6 result: (s = 5, a = :left, r = -1, sp = 4, t = 2)
print agent 7 result: (s = 5, a = :left, r = -1, sp = 4, t = 2)
print agent 8 result: (s = 7, a = :right, r = -1, sp = 8, t = 2)
print agent 9 result: (s = 5, a = :left, r = -1, sp = 4, t = 2)
print agent 10 result: (s = 5, a = :left, r = -1, sp = 4, t = 2)
t:2
print agent 1 result: (s = 4, a = :left, r = -1, sp = 5, t = 3)
print agent 2 result: (s = 4, a = :left, r = -1, sp = 3, t = 3)
print agent 3 result: (s = 4, a = :left, r = -1, sp = 3, t = 3)
print agent 4 result: (s = 8

t:3
print agent 1 result: (s = 3, a = :left, r = -1, sp = 2, t = 4)
print agent 2 result: (s = 3, a = :left, r = -1, sp = 4, t = 4)
print agent 3 result: (s = 5, a = :left, r = -1, sp = 4, t = 4)
print agent 4 result: (s = 3, a = :left, r = -1, sp = 2, t = 4)
print agent 5 result: (s = 5, a = :left, r = -1, sp = 4, t = 4)
print agent 6 result: (s = 3, a = :left, r = -1, sp = 4, t = 4)
print agent 7 result: (s = 5, a = :left, r = -1, sp = 4, t = 4)
print agent 8 result: (s = 3, a = :left, r = -1, sp = 2, t = 4)
print agent 9 result: (s = 3, a = :left, r = -1, sp = 2, t = 4)
print agent 10 result: (s = 3, a = :left, r = -1, sp = 4, t = 4)
t:4
print agent 1 result: (s = 2, a = :right, r = 10, sp = 1, t = 5)
print agent 2 result: (s = 4, a = :left, r = -1, sp = 3, t = 5)
print agent 3 result: (s = 4, a = :left, r = -1, sp = 3, t = 5)
print agent 4 result: (s = 2, a = :right, r = 10, sp = 1, t = 5)
print agent 5 result: (s = 4, a = :left, r = -1, sp = 3, t = 5)
print agent 6 result: (s = 4,

Excessive output truncated after 524289 bytes.

print agent 1 result: (s = 6, a = :left, r = -1, sp = 5, t = 1)
print agent 2 result: (s = 6, a = :left, r = -1, sp = 5, t = 1)
print agent 3 result: (s = 6, a = :left, r = -1, sp = 5, t = 1)
print agent 4 result: (s = 6, a = :left, r = -1, sp = 7, t = 1)
print agent 5 result: (s = 6, a = :left, r = -1, sp = 5, t = 1)
print agent 6 result: (s = 6, a = :left, r = -1, sp = 5, t = 1)
print agent 7 result: (s = 6, a = :left, r = -1, sp = 5, t = 1)
print agent 8 result: (s = 6, a = :left, r = -1, sp = 5, t = 1)
print agent 9 result: (s = 6, a = :left, r = -1, sp = 5, t = 1)
print agent 10 result: (s = 6, a = :left, r = -1, sp = 5, t = 1)
t:1
print agent 1 result: (s = 5, a = :left, r = -1, sp = 4, t = 2)
print agent 2 result: (s = 5, a = :left, r = -1, sp = 4, t = 2)
print agent 3 result: (s = 5, a = :left, r = -1, sp = 4, t = 2)
print agent 4 result: (s = 7, a = :right, r = -1, sp = 8, t = 2)
print agent 5 result: (s = 5, a = :left, r = -1, sp = 4, t = 2)
print agent 6 result: (s = 5, a = 

In [20]:
# paramter for seed sampling
using Distributions
theta = 10 * sign(rand(Bernoulli(0.5))-.5)
ntheta = - theta

# Thompson sampling

10.0